In [1]:
import requests
import config
import json
import os
import urllib.request
import planet

In [2]:
## variables to change
PLANET_API_KEY = 'ba80d8f478df4ec6ba4247ca3c9c8112' # <= insert API key here 
mosaic_date = "2020-09" ## year and month for basemap
DIR = '/Users/michaelcecil/Rprojects/geog287387_s/materials/data/assignment02' ## (change to your output dir)
# set your own bounding box
# use format below (lowerx,lowery,upperx,uppery)
string_bbox = "-71.78366,2.27885,-71.78366,2.27885" 

In [3]:
#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"
#setup session
session = requests.Session()
#authenticate
session.auth = (PLANET_API_KEY, "") 

In [4]:
mosaic_name = "planet_medres_normalized_analytic_" + mosaic_date + "_mosaic"

parameters = {
    "name__is" : mosaic_name # <= customize to your use case
}
#make get request to access mosaic from basemaps API
res = session.get(API_URL, params = parameters)
print("status code:", res.status_code) #response status code


#print metadata for mosaic
mosaic = res.json()
print("mosaic metadata:", json.dumps(mosaic, indent=2))

status code: 200
mosaic metadata: {
  "_links": {
    "_self": "https://api.planet.com/basemaps/v1/mosaics?api_key=ba80d8f478df4ec6ba4247ca3c9c8112"
  },
  "mosaics": [
    {
      "_links": {
        "_self": "https://api.planet.com/basemaps/v1/mosaics/755805d4-8eba-4a61-a6ab-7514c1bde810?api_key=ba80d8f478df4ec6ba4247ca3c9c8112",
        "quads": "https://api.planet.com/basemaps/v1/mosaics/755805d4-8eba-4a61-a6ab-7514c1bde810/quads?api_key=ba80d8f478df4ec6ba4247ca3c9c8112&bbox={lx},{ly},{ux},{uy}",
        "tiles": "https://tiles.planet.com/basemaps/v1/planet-tiles/planet_medres_normalized_analytic_2020-09_mosaic/gmap/{z}/{x}/{y}.png?api_key=ba80d8f478df4ec6ba4247ca3c9c8112"
      },
      "bbox": [
        -179.999999974944,
        -30.145127179527,
        179.999999975056,
        30.145127179625
      ],
      "coordinate_system": "EPSG:3857",
      "datatype": "uint16",
      "first_acquired": "2020-09-01T00:00:00.000Z",
      "grid": {
        "quad_size": 4096,
        "resol

In [5]:

mosaic_id = mosaic['mosaics'][0]['id'] #get mosaic id
print('Mosaic id: '+ mosaic_id)

#get bbox for entire mosaic
mosaic_bbox = mosaic['mosaics'][0]['bbox']
print("mosaic bbox: ", mosaic_bbox)
mosaic_bbox


#convert mosaic bbox to string for search params
#string_bbox = ','.join(map(str, mosaic_bbox))
#print(string_bbox)



print("string bbox:", string_bbox)


Mosaic id: 755805d4-8eba-4a61-a6ab-7514c1bde810
mosaic bbox:  [-179.999999974944, -30.145127179527, 179.999999975056, 30.145127179625]
string bbox: -71.78366,2.27885,-71.78366,2.27885


In [6]:
#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}

#accessing quads using metadata from mosaic
quads_url = "{}/{}/quads".format(API_URL, mosaic_id)
res = session.get(quads_url, params=search_parameters, stream=True)
quads = res.json()
items = quads['items']
print(items)

[{'_links': {'download': 'https://api.planet.com/basemaps/v1/mosaics/755805d4-8eba-4a61-a6ab-7514c1bde810/quads/615-1036/full?api_key=ba80d8f478df4ec6ba4247ca3c9c8112'}, 'bbox': [-71.89453125, 2.1088986592431382, -71.71875, 2.28455066023697], 'id': '615-1036', 'percent_covered': None}]


In [14]:


#iterate over quad download links and save to folder by id
for i in items:
    link = i['_links']['download']
    name = i['id'] + "_" + mosaic_date
    name = name + '.tiff'
    filename = os.path.join(DIR, name)

    #checks if file already exists before saving
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(link, filename)